In [1]:
import os
import PyPDF2

def convert_pdf_to_txt(pdf_path, txt_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ''
            for page in pdf_reader.pages:
                text += page.extract_text()
            
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)
            print(f"PDF '{pdf_path}' converted to TXT successfully!")
    except Exception as e:
        print(f"Error converting '{pdf_path}':", e)

def batch_convert_pdfs(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for pdf_file in os.listdir(input_folder):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(input_folder, pdf_file)
            txt_filename = os.path.splitext(pdf_file)[0] + '.txt'
            txt_path = os.path.join(output_folder, txt_filename)
            convert_pdf_to_txt(pdf_path, txt_path)


In [2]:
# Replace 'input_folder' with the folder containing your PDF files
input_folder = 'pdf_data'

# Replace 'output_folder' with the desired folder for the output TXT files
output_folder = 'txt_data'

batch_convert_pdfs(input_folder, output_folder)

PDF 'pdf_data\ChatBot1 english 2.pdf' converted to TXT successfully!
PDF 'pdf_data\FAQ 2.pdf' converted to TXT successfully!


In [3]:

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [24]:
# loader = DirectoryLoader("./txt_data", glob="*.txt", loader_cls=TextLoader)
loader1 = DirectoryLoader('txt_data/', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'autodetect_encoding': True})

splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                       chunk_overlap=50)
documents1 = loader1.load()
texts = splitter.split_documents(documents1)

In [20]:
# Direct Pdf File directory Loading


# from langchain_community.document_loaders import PyPDFDirectoryLoader
# loader = PyPDFDirectoryLoader("pdf_data/")
# # interpret information in the documents
# documents =  loader.load()

# splitter = RecursiveCharacterTextSplitter(chunk_size=500,
#                                           chunk_overlap=50)
# texts = splitter.split_documents(documents)

In [29]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

c:\Users\mpasha1\OneDrive - Insight\Desktop\Llama 2 work\Llama2 em\llmavecenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\mpasha1\OneDrive - Insight\Desktop\Llama 2 work\Llama2 em\llmavecenv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mpasha1\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-t

In [30]:
# create and save the local database
db = FAISS.from_documents(texts, embeddings)
db.save_local("faiss")


In [31]:
from langchain.llms import CTransformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [32]:
# prepare the template we will use when prompting the AI
template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

# load the language model
llm = CTransformers(model='model/llama-2-7b-chat.ggmlv3.q8_0.bin',
                    model_type='llama',
                    config={'max_new_tokens': 256, 'temperature': 0.01})

In [33]:

# load the interpreted information from the local database
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})
db = FAISS.load_local("faiss", embeddings)

In [34]:
# prepare a version of the llm pre-loaded with the local content
retriever = db.as_retriever(search_kwargs={'k': 2})
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'])
qa_llm = RetrievalQA.from_chain_type(llm=llm,
                                     chain_type='stuff',
                                     retriever=retriever,
                                     return_source_documents=True,
                                     chain_type_kwargs={'prompt': prompt})

In [35]:
# ask the AI chat about information in our local files
prompt = "what is lamination?"
output = qa_llm({'query': prompt})
print(output["result"])

c:\Users\mpasha1\OneDrive - Insight\Desktop\Llama 2 work\Llama2 em\llmavecenv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Lamination involves the application of a range of papers available in white, ivory or pearl to protect the print from rubbing and make it tear-proof and resistant to chemical agents. It makes labels stand out and is especially suitable for high-end containers or luxury products.


In [36]:
# ask the AI chat about information in our local files
prompt = " Do you print with variable data?"
output = qa_llm({'query': prompt})
print(output["result"])

Yes, we use Mosaic and Collage technologies for variable data printing.


In [38]:
# ask the AI chat about information in our local files
prompt = " In my file, there are Pantone colors. Will I actually get the same reproduction during printing?"
output = qa_llm({'query': prompt})
print(output["result"])

Our printing systems allow us to be faithful to Pantone colors with a precision range that goes from 95% to 100%. To find out if your Pantone color falls within this range, contact us at info@auroflex.it


In [40]:
# ask the AI chat about information in our local files
prompt = " What characteristics should the file to be sent have?"
output = qa_llm({'query': prompt})
print(output["result"])

The ideal formats for sending graphic files are PDF and AI. The label must be in CMYK four-color process. All RGB projects will be converted. Yes, the fonts should be converted into paths. Yes, the images should be embedded in the document with a resolution of 300dpi. No, you cannot overlap more than 4 inks.


In [41]:
# ask the AI chat about information in our local files
prompt = " What characteristics should the file to be sent have?"
output = qa_llm({'query': prompt})
print(output["result"])

The ideal formats for sending graphic files are PDF and AI. The label must be in CMYK four-color process. All RGB projects will be converted. Yes, the fonts should be converted into paths. Yes, the images should be embedded in the document with a resolution of 300dpi. No, you cannot overlap more than 4 inks.


In [ ]:
"""It must be a PDF file with 300dpi images embedded and fonts converted into 
paths, the color space must be CMYK and in case of using Pantone colors you should 
use the correct version in relation to the support on which you intend to print. U 
for Natural papers, C for Coated papers and Plastic Films"""

In [42]:
# ask the AI chat about information in our local files
prompt = " What is the advantages of lamination?"
output = qa_llm({'query': prompt})
print(output["result"])

Lamination (lamination) of the label has many advantages: it protects the print from rubbing and makes the label tear -proof and resistant to chemical agents. It makes your labels stand out, making them especially suitable for high -end containers or luxury products.


In [43]:
# ask the AI chat about information in our local files
prompt = " What is the advantages of lamination? and give me in point vices?"
output = qa_llm({'query': prompt})
print(output["result"])

Lamination (lamination) of the label has many advantages: it protects the print from rubbing and makes 
the label tear -proof and resistant to chemical agents. It makes your labels stand out, making them 
especially suitable for high -end containers or luxury products. Laminated materials can produce unique 
effects, from elegant matte to silky.

Please let me know if you need anything else.


In [44]:
# ask the AI chat about information in our local files
prompt = " Lamination ke advantages kya hai?"
output = qa_llm({'query': prompt})
print(output["result"])

Lamination provides protection to the print from rubbing and makes it tear-proof and resistant to chemical agents. It also makes labels stand out, making them especially suitable for high-end containers or luxury products.


In [45]:
# ask the AI chat about information in our local files
prompt = " How is the prime minster of india?"
output = qa_llm({'query': prompt})
print(output["result"])

The maximum file size that can be printed is 20 MB.
